In [1]:
import os
import pandas as pd
import altair as alt

In [2]:
#os.chdir("/content/drive/MyDrive/Harvey Associate Initiative/")
#os.chdir("/Users/rerobbins/projects/work/harvey_training_initiative")
os.chdir("/home/jovyan/work/work/harvey_training_initiative")

In [3]:
df = pd.read_excel("training_status_directory.xlsx")
df = df[(df['Title Group'] == "Associate") & (df['Region'] != "U.S.")]

In [4]:
grouped = df.groupby(['Region', 'Office', 'Trained']).size().reset_index(name='Count')

In [5]:
pivot = grouped.pivot(index=['Region', 'Office'], columns='Trained', values='Count').reset_index()
pivot.columns = ['Region', 'Office', 'UntrainedCount', 'TrainedCount']
pivot = pivot.fillna(0)

pivot['Total'] = pivot['TrainedCount'] + pivot['UntrainedCount']

In [6]:
stacked_data_office = pivot.melt(
    id_vars=['Region', 'Office', 'Total'],
    value_vars=['TrainedCount', 'UntrainedCount'],
    var_name='TrainingStatus',
    value_name='Count'
)

stacked_data_office['TrainingStatus'] = pd.Categorical(
    stacked_data_office['TrainingStatus'].map({
        'UntrainedCount': 'Untrained',
        'TrainedCount': 'Trained'
    }),
)

In [7]:
region_data = pivot.groupby('Region').agg(
    Total=('Total', 'sum'),
    TrainedCount=('TrainedCount', 'sum'),
    UntrainedCount=('UntrainedCount', 'sum')
).reset_index()

stacked_data_region = region_data.melt(
    id_vars=['Region', 'Total'],
    value_vars=['TrainedCount', 'UntrainedCount'],
    var_name='TrainingStatus',
    value_name='Count'
)

stacked_data_region['TrainingStatus'] = pd.Categorical(
    stacked_data_region['TrainingStatus'].map({
        'TrainedCount': 'Trained',
        'UntrainedCount': 'Untrained'
    }),
)

In [8]:
def create_region_chart(region):
    filtered_data = stacked_data_office[stacked_data_office['Region'] == region]
    return alt.Chart(filtered_data).mark_bar().encode(
        x=alt.X('Office:N', title='Office'),
        y=alt.Y('Count:Q',  title='Number of People', stack=True),
        color=alt.Color(
            'TrainingStatus:N',
            scale=alt.Scale(
                domain=['Untrained', 'Trained'],
                range=['red', 'green']
            ),
            title='Training Status'
        ),
        tooltip=[
            alt.Tooltip('Region:N', title='Region'),
            alt.Tooltip('Office:N', title='Office'),
            alt.Tooltip('Total:Q', title='Total People'),
            alt.Tooltip('TrainingStatus:N', title='Status'),
            alt.Tooltip('Count:Q', title='Count')
        ]
    ).properties(
        title=f'{region} Region',
        width=300,
        height=200
    )

In [9]:
region_chart = alt.Chart(stacked_data_region).mark_bar().encode(
    x=alt.X('Region:N', title='Region'),
    y=alt.Y('Count:Q', title='Number of People', stack=True),
    color=alt.Color(
        'TrainingStatus:N',
        scale=alt.Scale(
            domain=['Untrained', 'Trained'],
            range=['red', 'green']
        ),
        title='Training Status'
    ),
    tooltip=[
        alt.Tooltip('Region:N', title='Region'),
        alt.Tooltip('Total:Q', title='Total People'),
        alt.Tooltip('TrainingStatus:N', title='Status'),
        alt.Tooltip('Count:Q', title='Count')
    ]
).properties(
    title='Region-Level Summary: Training Progress',
    width=600,
    height=300
)

In [10]:
# Create individual charts for each region
regions = stacked_data_office['Region'].unique()
charts = [create_region_chart(region) for region in regions]

In [11]:
# Combine the region-specific charts
office_charts = alt.hconcat(*charts).properties(
    title='Office-Level Detail: Training Progress by Region'
)

In [12]:
# Final Combined Chart
final_chart = alt.vconcat(region_chart, office_charts).properties(
    title='Training Progress Overview'
)

In [13]:
final_chart

alt.VConcatChart(...)

In [14]:
final_chart.save("emea.png")

ValueError: Saving charts in 'png' format requires the vl-convert-python package: see https://altair-viz.github.io/user_guide/saving_charts.html#png-svg-and-pdf-format